# MLPClassifier in SciKeras

SciKeras is a bridge between Keras and Scikit-Learn. As such, one of SciKeras' design goals is to be able to create a Scikit-Learn style estimator backed by Keras. 

This notebook implements an estimator that is analogous to `sklearn.neural_network.MLPClassifier` using Keras. This estimator should (for the most part) work as a drop-in replacement for `MLPClassifier`!

<table align="left"><td>
<a target="_blank" href="https://colab.research.google.com/github/adriangb/scikeras/blob/master/notebooks/Basic_Usage.ipyn">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>  
</td><td>
<a target="_blank" href="https://github.com/adriangb/scikeras/blob/master/notebooks/Basic_Usage.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a></td></table>

### Table of contents

* [Defining the Keras Model](#Defining-the-Keras-Model)
  * [Inputs](#Inputs)
  * [Hidden layers](#Hidden-layers)
  * [Output layers](#Training-the-neural-net-classifier)
  * [Losses and optimizer](#Losses-and-optimizer)
  * [Wrapping with SciKeras](#Wrapping-with-scikeras)
* [Testing Classifier](#Testing-our-classifier)
* [Self contained classifier](#Self-contained-classifier)
  * [Subclassing](#Subclassing)

Install SciKeras

In [4]:
!python -m pip install git+https://github.com/adriangb/scikeras.git@master

  Cloning https://github.com/adriangb/scikeras.git (to revision master) to /private/var/folders/qx/54cwp9996s77rb4ng5j8y_v8zmmjlk/T/pip-req-build-y54w13qa
  Running command git clone -q https://github.com/adriangb/scikeras.git /private/var/folders/qx/54cwp9996s77rb4ng5j8y_v8zmmjlk/T/pip-req-build-y54w13qa
  Installing build dependencies ... ^C
canceled
ERROR: Operation cancelled by user
You should consider upgrading via the '/Users/adrian.badaracco/Documents/GitHub/scikeras/.venv/bin/python -m pip install --upgrade pip' command.


Silence TensorFlow logging to keep output succint.

In [5]:
from tensorflow import get_logger
get_logger().setLevel('ERROR')

In [18]:
from typing import Dict, Iterable, Any

from scikeras.wrappers import KerasClassifier, KerasRegressor
from tensorflow import keras

## Defining the Keras Model

First, we outline our model building function, using a `Sequential` Model:

In [7]:
def get_clf_model():
    model = keras.Sequential()
    return model

### Inputs

We need to define an input layer for Keras. SciKeras allows you to dynamically determine the input size based on the features (`X`). To do this, you need to add the `meta` parameter to `get_clf_model`'s parameters. `meta` will be a dictionary with all of the `meta` attributes that `KerasClassifier` generates during the `fit` call, including `n_features_in_`, which we will use to dynamically size the input layer.

In [4]:
def get_clf_model(meta: dict):
    model = keras.Sequential()
    inp = keras.layers.Input(shape=(meta["n_features_in_"]))
    model.add(inp)
    return model

### Hidden Layers

Multilayer perceptrons are generally composed of an input layer, an output layer and 0 or more hidden layers. The size of the hidden layers is specified via the `hidden_layer_sizes` parameter in MLClassifier, where the the ith element represents the number of neurons in the ith hidden layer. Let's add that parameter:

In [7]:
def get_clf_model(hidden_layer_sizes: Iterable[int], meta: dict):
    model = keras.Sequential()
    inp = keras.layers.Input(shape=(meta["n_features_in_"]))
    model.add(inp)
    for hidden_layer_size in hidden_layer_sizes:
        layer = keras.layers.Dense(hidden_layer_size, activation="relu")
        model.add(layer)
    return model

### Output layers

The output layer needs to reflect the type of classification task being performed. Here, we will handle 2 cases:
* binary classification: single output unit with sigmoid activation
* multiclass classification: one output unit for each class, with softmax activation
The main complication arises from determining which one to use. Like with the input features, SciKeras provides useful information on the target within the `meta` parameter. Specifically, we will use the `n_classes_` and `target_type_` attributes to determine the number of output units and activation function.

In [12]:
def get_clf_model(hidden_layer_sizes: Iterable[int], meta: dict):
    model = keras.Sequential()
    inp = keras.layers.Input(shape=(meta["n_features_in_"]))
    model.add(inp)
    for hidden_layer_size in hidden_layer_sizes:
        layer = keras.layers.Dense(hidden_layer_size, activation="relu")
        model.add(layer)
    if meta["target_type_"] == "binary":
        n_output_units = 1
        output_activation = "sigmoid"
    elif meta["target_type_"] == "multiclass":
        n_output_units = meta["n_classes_"]
        output_activation = "softmax"
    else:
        raise NotImplementedError(f"Unsupported task type: {meta['target_type_']}")
    out = keras.layers.Dense(n_output_units, activation=output_activation)
    return model

For now, we raise a `NotImplementedError` for other target types. For an example handling multi-output target types, see the [Multi Output notebook](https://colab.research.google.com/github/adriangb/scikeras/blob/master/notebooks/MultiInput.ipynb).

### Losses and optimizer

Like the output layer, the loss must match the type of classification task. Generally, it is easier and safet to allow SciKeras to compile your model for you by passing the loss to `KerasClassifier` directly (`KerasClassifier(loss="binary_crossentropy")`). However, in order to implement custom logic around the choice of loss function, we compile the model ourselves within `get_clf_model`; SciKeras will not re-compile the model.

In [14]:
def get_clf_model(hidden_layer_sizes: Iterable[int], meta: dict):
    model = keras.Sequential()
    inp = keras.layers.Input(shape=(meta["n_features_in_"]))
    model.add(inp)
    for hidden_layer_size in hidden_layer_sizes:
        layer = keras.layers.Dense(hidden_layer_size, activation="relu")
        model.add(layer)
    if meta["target_type_"] == "binary":
        n_output_units = 1
        output_activation = "sigmoid"
        loss = "binary_crossentropy"
    elif meta["target_type_"] == "multiclass":
        n_output_units = meta["n_classes_"]
        output_activation = "softmax"
        loss = "sparse_categorical_crossentropy"
    else:
        raise NotImplementedError(f"Unsupported task type: {meta['target_type_']}")
    out = keras.layers.Dense(n_output_units, activation=output_activation)
    model.compile(loss=loss)
    return model

At this point, we have a valid, compiled model. However if we want to be able to tune the optimizer, we should accept `compile_kwargs` as a parameter in `get_clf_model`. `compile_kwargs` will be a dictionary containing valid `kwargs` for `Model.compile`, so we can unpack it directly like `model.compile(**compile_kwargs)`. In this case however, we will only be taking the `optimizer` kwarg.

In [31]:
def get_clf_model(hidden_layer_sizes: Iterable[int], meta: Dict[str, Any], compile_kwargs: Dict[str, Any]):
    model = keras.Sequential()
    inp = keras.layers.Input(shape=(meta["n_features_in_"]))
    model.add(inp)
    for hidden_layer_size in hidden_layer_sizes:
        layer = keras.layers.Dense(hidden_layer_size, activation="relu")
        model.add(layer)
    if meta["target_type_"] == "binary":
        n_output_units = 1
        output_activation = "sigmoid"
        loss = "binary_crossentropy"
    elif meta["target_type_"] == "multiclass":
        n_output_units = meta["n_classes_"]
        output_activation = "softmax"
        loss = "sparse_categorical_crossentropy"
    else:
        raise NotImplementedError(f"Unsupported task type: {meta['target_type_']}")
    out = keras.layers.Dense(n_output_units, activation=output_activation)
    model.compile(loss=loss, optimizer=compile_kwargs["optimizer"])
    return model

### Wrapping with SciKeras

Our last step in defining our model is to wrap it with SciKeras. A couple of things to note are:
* Every user-defined parameter in `model`/`get_clf_model` (in our case just `hidden_layer_sizes`) must be defined as a keyword argument to `KerasClassifier` with a default value.
* Keras defaults to `"rmsprop"` for `optimizer`. We set it to `"adam"` to mimic MLPClassifier.
* We set the learning rate for the optimizer to `0.001`, again to mimic MLPClassifier. We set this parameter using [parameter routing](https://scikeras.readthedocs.io/en/latest/advanced.html#routed-parameters).
* Other parameters, such as `activation`, can be added similar to `hidden_layer_sizes`, but we omit them here for simplicity.

In [32]:
clf = KerasClassifier(
    model=get_clf_model,
    hidden_layer_sizes=(100, ),
    optimizer="adam",
    optimizer__learning_rate=0.001,
)

## Testing our classifier

Before continouing, we will run a small test to make sure we get somewhat reasonable results.

In [33]:
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score

In [36]:
X, y = make_classification()

# check that fit works
clf.fit(X, y)
clf.predict(X)
# check cross-validation score
# cross_val_score(clf, X, y)

4/4 [==============================] - 0s 982us/step


ValueError: y should be a 1d array, got an array of shape (100, 100) instead.

Also, as in `sklearn`, you may call `predict` or `predict_proba` on the fitted model.

### Making predictions, classification

In [12]:
y_pred = clf.predict(X[:5])
y_pred

1/1 [==============================] - 0s 2ms/step


array([0, 0, 0, 0, 0])

In [13]:
y_proba = clf.predict_proba(X[:5])
y_proba

1/1 [==============================] - 0s 2ms/step


array([[0.5358717 , 0.46412832],
       [0.6633073 , 0.3366927 ],
       [0.71149606, 0.28850394],
       [0.8953113 , 0.1046887 ],
       [0.7850856 , 0.21491438]], dtype=float32)

## Training a regressor

### A toy regression task

In [14]:
from sklearn.datasets import make_regression

In [15]:
X_regr, y_regr = make_regression(1000, 20, n_informative=10, random_state=0)

In [16]:
X_regr.shape, y_regr.shape, y_regr.min(), y_regr.max()

((1000, 20), (1000,), -649.0148244404172, 615.4505181286091)

### Definition of the `Keras` regression `Model`

Again, define a vanilla neural network. The main difference is that the output layer always has a single unit and does not apply any nonlinearity.

In [17]:
def get_reg(meta, hidden_layer_sizes, dropout):
    n_features_in_ = meta["n_features_in_"]
    model = keras.models.Sequential()
    model.add(keras.layers.Input(shape=(n_features_in_,)))
    for hidden_layer_size in hidden_layer_sizes:
        model.add(keras.layers.Dense(hidden_layer_size, activation="relu"))
        model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Dense(1))
    return model

### Defining and training the neural net regressor

Training a regressor is almost the same as training a classifier. Mainly, we use `KerasRegressor` instead of `KerasClassifier` (this is the same terminology as in `sklearn`). We also change the loss function to `KerasRegressor.r_squared`. SciKeras provides this loss function because most of the `sklearn` ecosystem expects `R^2` as the loss function, but Keras does not have a default implementation.

In [18]:
from scikeras.wrappers import KerasRegressor

In [19]:
reg = KerasRegressor(
    model=get_reg,
    loss=KerasRegressor.r_squared,
    optimizer="adam",
    hidden_layer_sizes=(100,),
    dropout=0.5,
)

In [20]:
reg.fit(X_regr, y_regr)

32/32 [==============================] - 0s 1ms/step - loss: -0.0247


KerasRegressor(
	model=<function get_reg at 0x7f998ba1f488>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=adam
	loss=<function KerasRegressor.r_squared at 0x7f998c756400>
	metrics=None
	batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	hidden_layer_sizes=(100,)
	dropout=0.5
)

### Making predictions, regression

You may call `predict` or `predict_proba` on the fitted model. For regressions, both methods return the same value.

In [21]:
y_pred = reg.predict(X_regr[:5])
y_pred

1/1 [==============================] - 0s 1ms/step


array([ 0.16403273, -0.36295909,  0.50139022, -0.062492  , -0.42927906])

## Saving and loading a model

Save and load either the whole model by using pickle, or use Keras' specialized save methods on the `KerasClassifier.model_` or `KerasRegressor.model_` attribute that is created after fitting. You will want to use Keras' model saving utilities if any of the following apply:
1. You wish to save only the weights or only the training configuration of your model.
2. You wish to share your model with collaborators. Pickle is a relatively unsafe protocol and it is not recommended to share or load pickle objects publically.
3. You care about performance, especially if doing in-memory serialization.

For more information, see Keras' [saving documentation](https://www.tensorflow.org/guide/keras/save_and_serialize).

### Saving the whole model

In [22]:
import pickle

In [23]:
bytes_model = pickle.dumps(reg)

In [24]:
new_reg = pickle.loads(bytes_model)
new_reg

KerasRegressor(
	model=<function get_reg at 0x7f998ba1f488>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=adam
	loss=<function KerasRegressor.r_squared at 0x7f998c756400>
	metrics=None
	batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	hidden_layer_sizes=(100,)
	dropout=0.5
)

### Saving using Keras' saving methods

This efficiently and safely saves the model to disk, including trained weights.


In [25]:
# Save to disk
reg.model_.save("/tmp/my_model")

In [26]:
# Load the model back into memory
new_reg_model = keras.models.load_model("/tmp/my_model")
# Now we need to instantiate a new SciKeras object with this model
# Note that we no longer pass paramters like hidden_layer_sizes, those
# are note "fixed"
reg = KerasRegressor(
    new_reg_model,
    loss=KerasRegressor.r_squared,
    optimizer="adam",
)
reg.fit(X_regr, y_regr)
reg.predict(X_regr[:5])

1/1 [==============================] - 0s 1ms/step


array([-0.04418676, -0.11787903,  0.56504041, -0.10095135, -0.58096522])

## Usage with an `sklearn Pipeline`

It is possible to put the `KerasClassifier` inside an `sklearn Pipeline`, as you would with any `sklearn` classifier.

In [27]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [28]:
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('clf', clf),
])

In [29]:
pipe.fit(X, y)

32/32 [==============================] - 0s 1ms/step - loss: 0.7432


Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('clf',
                 KerasClassifier(batch_size=None, build_fn=None, callbacks=None, dropout=0.5, epochs=1, hidden_layer_sizes=(100,), loss='binary_crossentropy', metrics=None, model=<function get_clf at 0x7f998ba1f268>, optimizer='adam', random_state=None, run_eagerly=False, shuffle=True, validation_split=0.0, verbose=1, warm_start=False))],
         verbose=False)

In [30]:
y_proba = pipe.predict_proba(X[:5])
y_proba

1/1 [==============================] - 0s 2ms/step


array([[0.5154557 , 0.48454428],
       [0.55230534, 0.44769466],
       [0.48315734, 0.51684266],
       [0.6526959 , 0.34730408],
       [0.63546836, 0.3645316 ]], dtype=float32)

To save the whole pipeline, including the Keras model, use `pickle`.

## Callbacks

Adding a new callback to the model is straightforward. Below we show how to add an `EarlyStopping` callback to prevent overfitting.

In [31]:
es = keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', mode='max', patience=200, verbose=1)

We now generate a toy dataset using `sklearn.datasets.make_moons`. This dataset was chosen specifically to trigger early stopping.

In [32]:
from sklearn.datasets import make_moons

In [33]:
X, y = make_moons(n_samples=100, noise=0.2, random_state=0)
X.shape, y.shape

((100, 2), (100,))

We will first check fitting without the callback and then with. We will compare the training time and final accuracy.

In [34]:
import time

In [36]:
# First test without the callback
clf = KerasClassifier(
    model=get_clf,
    loss="binary_crossentropy",
    optimizer="adam",
    hidden_layer_sizes=(500,),
    dropout=0.5,
    metrics=["binary_accuracy"],
    fit__validation_split=0.2,
    epochs=500,
    verbose=False,
)
start = time.time()
clf.fit(X, y)
print(f"Training time: {time.time() - start}")
print(f"Final accuracy: {clf.history_['val_binary_accuracy'][-1][-1]}")  # get last value of last fit/partial_fit call

Training time: 10.489465475082397
Final accuracy: 1.0


In [37]:
# Test with the callback
clf = KerasClassifier(
    model=get_clf,
    loss="binary_crossentropy",
    optimizer="adam",
    hidden_layer_sizes=(500,),
    dropout=0.5,
    metrics=["binary_accuracy"],
    fit__validation_split=0.2,
    epochs=500,
    verbose=False,
    callbacks=[es]
)
start = time.time()
clf.fit(X, y)
print(f"Training time: {time.time() - start}")
print(f"Final accuracy: {clf.history_['val_binary_accuracy'][-1][-1]}")  # get last value of last fit/partial_fit call

Epoch 00215: early stopping
Training time: 4.222731351852417
Final accuracy: 0.949999988079071


For information on how to write custom callbacks, have a look at the 

---

[Advanced_Usage](https://nbviewer.jupyter.org/github/adriangb/scikeras/blob/master/notebooks/Advanced_Usage.ipynb) notebook.

## Usage with sklearn `GridSearchCV`

### Special prefixes

SciKeras allows to direct access to all parameters passed to the wrapper constructors, including deeply nested routed parameters. This allows tunning of
paramters like `hidden_layer_sizes` as well as `optimizer__learning_rate`.

This is exactly the same logic that allows to access estimator parameters in `sklearn Pipeline`s and `FeatureUnion`s.

This feature is useful in several ways. For one, it allows to set those parameters in the model definition. Furthermore, it allows you to set parameters in an `sklearn GridSearchCV` as shown below.

To differentiate paramters like `callbacks` which are accepted by both `tf.keras.Model.fit` and `tf.keras.Model.predict` you can add a `fit__` or `predict__` routing suffix respectively. Similar, the `model__` prefix may be used to specify that a paramter is destined only for `get_clf`/`get_reg` (or whatever callable you pass as your `model` argument).

For more information on parameter routing with special prefixes, see the [Advanced Usage Docs](https://scikeras.org.readthedocs.build/en/latest/advanced.html#routed-parameters)

### Performing a grid search

Below we show how to perform a grid search over the learning rate (`optimizer__lr`), the model's number of hidden layers (`model__hidden_layer_sizes`), the model's dropout rate (`model__dropout`).

In [38]:
from sklearn.model_selection import GridSearchCV

In [39]:
clf = KerasClassifier(
    model=get_clf,
    loss="binary_crossentropy",
    optimizer="adam",
    optimizer__lr=0.1,
    model__hidden_layer_sizes=(100,),
    model__dropout=0.5,
    verbose=False,
)

*Note*: We set the verbosity level to zero (`verbose=False`) to prevent too much print output from being shown.

In [40]:
params = {
    'optimizer__lr': [0.05, 0.1],
    'model__hidden_layer_sizes': [(100, ), (50, 50, ), (33, 33, 33, )],
    'model__dropout': [0, 0.5],
}

In [51]:
gs = GridSearchCV(clf, params, scoring='accuracy', n_jobs=-1, verbose=True)

In [52]:
gs.fit(X, y)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   18.5s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   24.9s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=KerasClassifier(batch_size=None, build_fn=None, callbacks=None, epochs=1, loss='binary_crossentropy', metrics=None, model=<function get_clf at 0x7f998ba1f268>, model__dropout=0.5, model__hidden_layer_sizes=(100,), optimizer='adam', optimizer__lr=0.1, random_state=None, run_eagerly=False, shuffle=True, validation_split=0.0, verbose=False, warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'model__dropout': [0, 0.5],
                         'model__hidden_layer_sizes': [(100,), (50, 50),
                                                       (33, 33, 33)],
                         'optimizer__lr': [0.05, 0.1]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=True)

In [53]:
print(gs.best_score_, gs.best_params_)

0.86 {'model__dropout': 0, 'model__hidden_layer_sizes': (50, 50), 'optimizer__lr': 0.1}


Of course, we could further nest the `KerasClassifier` within an `sklearn Pipeline`, in which case we just prefix the parameter by the name of the net (e.g. `clf__model__hidden_layer_sizes`).